In [1]:
import os
import time
import cv2
import pickle
import datetime
from PIL import Image
from random import shuffle
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt

import keras.backend as K
import tensorflow as tf
from keras import regularizers, initializers
from keras.models import Model
from keras.optimizers import SGD, Adam, RMSprop
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger, LearningRateScheduler, TerminateOnNaN

from modelDesign import MNv2_segment_depth_multiloss_model
from utils_KITTI import *

Using TensorFlow backend.
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) /

## Parameters Setting

In [2]:
#是否要載入 pretrain weights
LOAD_DEPTH_WEIGHT = False


# 欲載入 pretrain weight 的路徑
load_depthWeight_path = ''

# 儲存 training weight 的路徑
save_depthWeight_path = '/home/islab/Documents/drone/depth_prediction/model/'

# Dataset路徑
airsim_depth_folder = '/home/islab/Documents/drone/depth_prediction/data/random'
# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# 設定 semantic segmentation class 數量 
CLASSES = 6

## Build Model

In [3]:
# 建立 Model layer 架構
model = MNv2_segment_depth_multiloss_model(inputShape=(INPUT_HEIGHT,INPUT_WIDTH,3), alpha=0.35, 
                                           expansion_factor=6, depth_multiplier=1, CLASSES=CLASSES,
                                           lock_backend_weights = False)

# 若LOAD_DEPTH_WEIGHT為True，載入Model weight
if LOAD_DEPTH_WEIGHT:
    model.load_weights(load_depthWeight_path)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 384, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 10, 12, 112) 261728      input_1[0][0]                    
__________________________________________________________________________________________________
subpixel_1 (Lambda)             (None, 20, 24, 28)   0           model_1[1][0]                    
__________________________________________________________________________________________________
conv_expand_1 (Conv2D)          (None, 20, 24, 168)  4704        subpixel_1[0][0]                 
____________________________________________________________________________________________

## Training

In [4]:
# 訓練的時間戳記
timestamp = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')

# 設定訓練參數的儲存路徑與方式
model_checkpoint = ModelCheckpoint(filepath=save_depthWeight_path + 'airsim_' + timestamp + '_epoch-{epoch:03d}_loss-{loss:.4f}.h5',
                                   monitor='loss',
                                   verbose=1,
                                   save_best_only=False,
                                   save_weights_only=True,
                                   mode='auto',
                                   period=1)

# 設定log檔的儲存方式
csv_logger = CSVLogger(filename=save_depthWeight_path + 'airsim_' + timestamp + '_training_log.csv',
                       separator=',',
                       append=True)

# 設定每個epoch的 learning rate
def lr_schedule(epoch):
    min = 0.001
    max = 0.01
    step = 20
    if epoch > step:
        return min
    else:
        return max-epoch*(max-min)/step
learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule)

# 設定當Loss為NaN時停止訓練
terminate_on_nan = TerminateOnNaN()

# 設定loss function的callback functions
callbacks = [model_checkpoint,
             csv_logger,
             learning_rate_scheduler,
             terminate_on_nan]

In [5]:
# 訓練起始 epoch [TODO]
initial_epoch = 0

# 訓練結束 epoch [TODO]
final_epoch = 50

# 每個 epoch 的 training iteration
steps_per_epoch = 1000

# 設定每個 loss output 的權重: [depth_pred_2x, depth_pred_4x, depth_pred_8x, depth_pred_16x,seg_pred_2x,seg_pred_4x,seg_pred_8x,seg_pred_16x]
loss_weights=[0.25,0.25,0.25,0.25,0,0,0,0,0]

# 建立 loss function
depth_loss = custom_depth_loss(depth_weight=1.0, disparity_weight=0.0)

# 載入所有 training data 的資料路徑，建立為 list
dataList = getAirsimPath(airsim_depth_folder)


# 設定 Training data
# @param: (list) dataList: training data 的資料路徑
# @param: (list) shape: 四個 output 的 resolution
# @param: (bool) include_sky: 若為 true，則在載入GT depth map過程中，會把 depth map 天空的部分的數值設為 max 
# @param: (bool) depth_crrection: 修正PSMNet depth map 的誤差
# @param: (bool) mixGroundTruth: 是否混和 PSMNet depth map 和 sparse depth map 的深度資訊做訓練
# @param: (int) batchSize: training batch size
# @param: (int) CLASSES: 語意分割的類別數量(預設為19)
# @param: (bool) log_depth: 是否將 GT depth map 做 log transform
# @param: (bool) random_crop: 是否對 depth map 進行 random crop，作為 data augmentation
# @return: (class) data_generator
data_generator = UltraHybridGenerator_multiloss(dataList, 
                                                shape=[(INPUT_HEIGHT//2,INPUT_WIDTH//2),(INPUT_HEIGHT//4,INPUT_WIDTH//4),(INPUT_HEIGHT//8,INPUT_WIDTH//8),(INPUT_HEIGHT//16,INPUT_WIDTH//16)],
                                                include_sky=False, depth_crrection=True,
                                                mixGroundTruth=False, batchSize = 5, CLASSES=CLASSES)

# 設定 loss function
model.compile(optimizer=Adam(), 
              loss=[depth_loss, depth_loss, depth_loss, depth_loss, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy],
              loss_weights=loss_weights)

# 開始訓練!!
history = model.fit_generator(generator=data_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              initial_epoch=initial_epoch)

Total images: 30000
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
1000/1000 [==============================] - 188s 188ms/step - loss: 11.6014 - depth_pw_4_loss: 11.1761 - depth_pw_3_loss: 11.0312 - depth_pw_2_loss: 11.4421 - depth_pw_1_loss: 12.6540 - activation_4_loss: 0.0035 - activation_3_loss: 0.0066 - activation_2_loss: 0.0034 - activation_1_loss: 0.0032 - dense_2_loss: 0.5245

Epoch 00001: saving model to /home/islab/Documents/drone/depth_prediction/model/airsim_2020-01-14-16-56_epoch-001_loss-11.6014.h5
Epoch 2/50
1000/1000 [==============================] - 189s 189ms/step - loss: 9.5820 - depth_pw_4_loss: 8.5564 - depth_pw_3_loss: 8.8615 - depth_pw_2_loss: 9.1990 - depth_pw_1_loss: 11.6426 - activation_4_loss: 0.0014 - activation_3_loss: 0.0015 - activation_2_loss: 0.0016 - activation_1_loss: 0.0017 - dense_2_loss: 0.4987

Epoch 00002: saving model to /home/islab/Documents/drone/depth_p

Epoch 35/50
1000/1000 [==============================] - 262s 262ms/step - loss: 7.3193 - depth_pw_4_loss: 7.0816 - depth_pw_3_loss: 6.9122 - depth_pw_2_loss: 7.3862 - depth_pw_1_loss: 7.8345 - activation_4_loss: 0.0012 - activation_3_loss: 0.0012 - activation_2_loss: 0.0011 - activation_1_loss: 9.7943e-04 - dense_2_loss: 0.4941

Epoch 00035: saving model to /home/islab/Documents/drone/depth_prediction/model/airsim_2020-01-14-16-56_epoch-035_loss-7.3193.h5
Epoch 36/50
1000/1000 [==============================] - 263s 263ms/step - loss: 7.3237 - depth_pw_4_loss: 7.0943 - depth_pw_3_loss: 6.9332 - depth_pw_2_loss: 7.3874 - depth_pw_1_loss: 7.8148 - activation_4_loss: 0.0013 - activation_3_loss: 0.0013 - activation_2_loss: 0.0012 - activation_1_loss: 0.0010 - dense_2_loss: 0.5030

Epoch 00036: saving model to /home/islab/Documents/drone/depth_prediction/model/airsim_2020-01-14-16-56_epoch-036_loss-7.3237.h5
Epoch 37/50
1000/1000 [==============================] - 262s 262ms/step - loss: 7